# PROYECTO 1 ALGEBRA LINEAL PARA CIENCIA DE DATOS (IMT 2210)
#### INTEGRANTES : 
#### - Benjamin Prieto
#### - Benjamin Varela
#### - Victor Bórquez

#### Primero importamos los modulos necesarios:

In [3]:
import pandas as pd
from vec import Vec

### A) Para cada usuario, crear su vector de gustos, utilizando la clase Vec. Donde el dominio son el id de las peliculas y la funcion movie.id-> rating (utilizando diccionario). Guardar los vectores en un diccionario users

Abrimos el archivo movies.csv para sacar los ids de todas las películas

In [ ]:
with open('movies.csv', 'r') as docu:
    arc = docu.readlines()
    #el len(arc) es uno mas que el ult id de pelicula
    lista_pelis = []
    for i in range(1, len(arc)):
        lista_pelis.append(i)
    dominio = set(lista_pelis)

Abrimos el archivo rating.csv para 

In [4]:
with open('ratings.csv', 'r') as file:
    a = file.readlines()

id_actual = 1
dicc = {}
users = {} 

for linea in a:
    linea = linea.strip('\n')
    id_us, id_pel, ratio = linea.split(',', 2)
    if id_us.isnumeric() == True:
        id_us = int(id_us)
        id_pel = int(id_pel)
        ratio = int(ratio)
        if id_us == id_actual:
            dicc[id_pel] = ratio
        else:
            vector = Vec(dominio, dicc)
            users[id_actual] = vector
            dicc = {}
            dicc[id_pel] = ratio
            id_actual = id_us
vector = Vec(dominio, dicc)
users[id_us] = vector
